In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop

import numpy as np
import random, sys
import pandas as pd


Using TensorFlow backend.


In [2]:
import os
print(os.listdir())

['.ipynb_checkpoints', 'dp.ipynb', 'd_code.csv', 'yang_30.txt', 'yang_40.txt']


In [3]:
df = pd.read_table('yang_30.txt', sep='\n', header=None)
df.columns = ['text']
df.head()

,text
0,S OL766 KG504 SF108 WR405 ET321 RD201 IO789 O...
1,S FG308 PO276 FG371 KL365 QW987 LO103 LO103 P...
2,S WD675 RE385 PK765 GH791 KG341 ZA107 WR896 O...
3,S RE385 QO233 WY390 GH700 ZA107 KO109 WO378 O...
4,S QE931 QO102 GH791 SD398 ER389 QE277 OP345 G...


In [4]:
df = pd.read_table('yang_40.txt', sep='\n', header=None)
df.columns = ['text']
df.head()

,text
0,S RD201 PL221 JV765 WY390 WY504 E
1,S WD100 ER300 ET650 HK504 PK213 SD388 ET321 G...
2,S PO203 HD405 KP345 OK163 KG907 PO276 LO103 T...
3,S ER119 WR405 KP843 ER119 SD398 QO102 HK377 F...
4,S FG371 FG391 GH211 SF108 ER119 OP350 UY201 F...


In [5]:
dp_code = pd.read_csv('d_code.csv')
d_code = list(dp_code.d_code)
dp_code.head()

,d_code,price
0,KP345,71100
1,OK163,123000
2,QW987,3500
3,ER203,38800
4,OP167,68900


In [6]:
dp_code_dic = dict(zip(dp_code.d_code, dp_code.price))
# dp_code_dic

In [7]:
all_text = ''
for str in df.text:
    all_text = all_text + ' ' + str
    
text = all_text
print('훈련 데이터: ', len(text)) # 글자, 공백 포함

text_split = text.split(' ')
chars = sorted(list(set(text_split)))  #charts는 set으로 중복을 없앤 알파뱃 순서로 나열
print('질병코드의 종류:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

훈련 데이터:  952814
질병코드의 종류: 103


In [8]:
char_indices

{'': 0,
 'BN377': 1,
 'CV305': 2,
 'CV751': 3,
 'E': 4,
 'ER119': 5,
 'ER203': 6,
 'ER300': 7,
 'ER389': 8,
 'ER897': 9,
 'ET320': 10,
 'ET321': 11,
 'ET650': 12,
 'FG308': 13,
 'FG371': 14,
 'FG391': 15,
 'GH211': 16,
 'GH433': 17,
 'GH700': 18,
 'GH791': 19,
 'HD405': 20,
 'HD433': 21,
 'HK377': 22,
 'HK504': 23,
 'IO789': 24,
 'JV109': 25,
 'JV765': 26,
 'JV766': 27,
 'KG341': 28,
 'KG504': 29,
 'KG766': 30,
 'KG907': 31,
 'KL365': 32,
 'KO109': 33,
 'KP321': 34,
 'KP345': 35,
 'KP843': 36,
 'LO103': 37,
 'LO976': 38,
 'OK163': 39,
 'OK211': 40,
 'OL344': 41,
 'OL766': 42,
 'OP167': 43,
 'OP345': 44,
 'OP350': 45,
 'PK213': 46,
 'PK244': 47,
 'PK765': 48,
 'PL221': 49,
 'PL250': 50,
 'PL890': 51,
 'PO203': 52,
 'PO276': 53,
 'QA107': 54,
 'QE277': 55,
 'QE931': 56,
 'QO102': 57,
 'QO233': 58,
 'QO304': 59,
 'QO349': 60,
 'QO743': 61,
 'QW987': 62,
 'QY204': 63,
 'RD201': 64,
 'RE385': 65,
 'RE397': 66,
 'RE567': 67,
 'RE577': 68,
 'S': 69,
 'SD308': 70,
 'SD388': 71,
 'SD398': 72,
 

In [9]:
maxlen = 5
step = 1
sentences = []
next_chars = []

for i in range(0, len(text_split) - maxlen, step):
    sentences.append(' '.join(text_split[i: i + maxlen]))
    next_chars.append(text_split[i + maxlen])
    
print('학습할 질병패턴의 수:', len(sentences))
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence.split(' ')):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
print(sentences[:5])

학습할 질병패턴의 수: 180465
['  S RD201 PL221', ' S RD201 PL221 JV765', 'S RD201 PL221 JV765 WY390', 'RD201 PL221 JV765 WY390 WY504', 'PL221 JV765 WY390 WY504 E']


In [10]:
print('LSTM 모델을 만들어 본다....')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.015)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# 후보추출
def sample(preds, temp=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

LSTM 모델을 만들어 본다....


In [11]:
generated_all = []
for iteration in range(1, 8):
    print()
    print('반복번호 = ', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
    for diversity in [1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6]:
        print()
        print('추출옵션 : ', diversity)
        generated =''
        sentence ='S FG391'
        generated += sentence
        print('질병시작패턴 = "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(35):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence.split(' ')):
                x[0, t, char_indices[char]] = 1
            
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            
            generated += " " + next_char
            sentence = ' '.join(sentence.split(' ')[1:]) + ' ' + next_char
            sys.stdout.write(' ' + next_char)
            sys.stdout.flush()
            
            if next_char == 'E':
                generated_all.append(generated)
                break
        print()
            


반복번호 =  1
Epoch 1/1
180465/180465 [==============================] - 19s 103us/step - loss: 4.0466

추출옵션 :  1.4
질병시작패턴 = "S FG391"
S FG391 KG341 WR896 PK765 RE397 PK213 QA107 ER203 QY204 UY503 SD388 OK211 CV751 QO349 OL766 RD201 OP345 WR405 JV109 PK213 RE567 WR789 OP167 ZA107 TY211 KG766 E

추출옵션 :  1.6
질병시작패턴 = "S FG391"
S FG391 KP843 OP167 GH791 UY503 TY901 OK163 HD433 KP843 SD308 KG766 PK765 PL890 ER389 PL890 TR109 HD405 IO789 TR675 QO102 UY980 FG371 E

추출옵션 :  1.8
질병시작패턴 = "S FG391"
S FG391 RE397 WD599 WR789 SF108 ZA780 RE577 RE385 PK213 FG308 RD201 IO789 PL250 WO378 TY341 GH791 JV766 QE277 PK213 KG341 PK213 RE397 E

추출옵션 :  2.0
질병시작패턴 = "S FG391"
S FG391 GH700 WR789 OL344 SD398 WD599 WO378 PK213 PL250 ET650 GH211 WR789 RE577 WY107 ET650 SF108 TR675 KL365 WO188 HD405 TY211 WR321 RE385 ER119 KP321 RD201 CV751 ZA107 OP345 UY980 QO102 GH791 ET650 UY201 QE931 PL250

추출옵션 :  2.2
질병시작패턴 = "S FG391"
S FG391 OK163 QA107 WR321 WO900 QW987 TY211 KP321 UY503 QW987 QE931 TR675 LO976 UY503 QO10

In [12]:
avg_price_1year = 0
sum_price = 0 
for i , sent in enumerate(generated_all):
    print(i,sent)
    for str in sent.split(" ")[1:-1]:
        sum_price += dp_code_dic[str]
        
avg_price_1year = round(sum_price / (i+1) , -2) 

print('-'*100)
print('이 분은 앞으로 1년간 총',avg_price_1year,'원의 병원비가 들 예정입니다.')
print('-'*100)

0 S FG391 KG341 WR896 PK765 RE397 PK213 QA107 ER203 QY204 UY503 SD388 OK211 CV751 QO349 OL766 RD201 OP345 WR405 JV109 PK213 RE567 WR789 OP167 ZA107 TY211 KG766 E
1 S FG391 KP843 OP167 GH791 UY503 TY901 OK163 HD433 KP843 SD308 KG766 PK765 PL890 ER389 PL890 TR109 HD405 IO789 TR675 QO102 UY980 FG371 E
2 S FG391 RE397 WD599 WR789 SF108 ZA780 RE577 RE385 PK213 FG308 RD201 IO789 PL250 WO378 TY341 GH791 JV766 QE277 PK213 KG341 PK213 RE397 E
3 S FG391 GH433 WY107 KO109 WY390 E
4 S FG391 ET650 WD675 FG391 OK163 ET321 JV109 KL365 RE567 PK244 PK765 WO900 FG308 KG766 WD100 WO900 UY109 FG308 UY112 ET650 PO203 UY980 QW987 RE385 WY504 GH791 FG391 UY112 KL365 E
5 S FG391 PO276 E
6 S FG391 WD599 UY980 TY211 KP843 WY107 OP345 QO233 ER119 PL250 E
7 S FG391 WD675 TY341 SD388 PL221 FG371 OK163 KP843 QW987 GH211 WR789 UY503 UY112 QW987 FG371 FG391 E
8 S FG391 FG371 PK765 QA107 OK211 HK504 WO378 GH791 PK213 PL890 ET321 ET650 WY390 E
9 S FG391 FG391 E
10 S FG391 PK244 ET321 RE397 SF108 HD405 KP345 UY503 KL365